In [12]:
import numpy as np
import json_tricks

answer = {}

inputs = json_tricks.load('inputs/inputs.json')


# Matrix Product II

$$A_{5 \times 10} \  B_{10 \times 2}\ C_{2 \times 30}\ D_{30 \times 3}\ E_{3 \times 9}$$

1. What will be the shape of the resulting matrix?
2. How many multiplication of numbers are required at best?

In [13]:
answer['task1'] = {
    '1': (5, 9),
    '2': 424
}



# Numpy expression

Using Numpy, write a function that calculates the 
following expression:

$$\exp(A^T(B + 2C) + 3I) \mathbf x,$$

where $I$ is an identity matrix of the necessary shape.

In [14]:
def numpy_expression(A, B, C, x):
    M = B + 2 * C
    mat = A.T @ M
    if mat.shape[0] != mat.shape[1]:
        return np.full(x.shape, np.nan, dtype=np.float64)
    mat = mat + 3 * np.eye(mat.shape[0], dtype=mat.dtype)
    from scipy.linalg import expm
    exp_mat = expm(mat)
    res = exp_mat @ x
    return res.astype(np.float64)

In [15]:
answer['task2'] = []
for one_input in inputs['task2']:
    answer['task2'].append(numpy_expression(**one_input))

# Einstein's Rule

In *Tensor Algebra*, a direct generalization of the Linear Algebra to the case of $N$-dimentional tables called *tensors* (normal matrix), the Einstein's rule exists.

It works as follows: if you see a duplicating upper and lower index in the formula, that means, this index convolves.

For example, the following tensor expression, summation and matrix product are equivalent:

$$a_k^l b_l^m = \sum_{l=1}^L a_k^l b_l^m = AB$$

In this notation subscript means row index and superscript means column index.

<details>
<summary> Note </summary>

> [!NOTE]
> Also at some point it will be important to know that:
> * lower index represents a contravariant dimension of a
> tensor
> * upper index represents a covariant dimension 
> of tensor. But let us omit this part for now.

</details>

# Task

Calculate the following expression written using Einstein's 
rule:

$$a_k^m b_m^n c_n^o d_l^k$$

In [16]:
def einsteins_rule(A, B, C, D):
    res = np.einsum('km,mn,no,lk->lo', A, B, C, D)
    return res

In [17]:
answer['task3'] = []
for one_input in inputs['task3']:
    answer['task3'].append(einsteins_rule(**one_input))

# Diagonal Matrix Product

You are given two square matrices: $A$ and $D$, where $A$ is a 
full matrix and $D$ is a diagonal matrix:

$$
A = \begin{bmatrix}
- & \mathbf a_1 & - \\
& \vdots & \\
- & \mathbf a_N & - \\
\end{bmatrix}
$$

$$
D = \textrm{diag}(d_1, d_2, \dots, d_N) = \begin{bmatrix}
d_1 & & & & \\
& d_2 & & & \\
& & d_3 & & \\
& & & \ddots & \\
& & & & d_N 
\end{bmatrix}
$$

Write a program to calculate the result of $DA$ and $AD$ in 
the fastest possible way.

In [18]:
def diag_prod_DA(A, D):
    if D.ndim == 2:
        d = np.diag(D)
    else:
        d = D
    res = (d[:, None] * A)
    return res

def diag_prod_AD(A, D):
    if D.ndim == 2:
        d = np.diag(D)
    else:
        d = D
    res = (A * d)
    return res

In [19]:
answer['task4_1'] = []
answer['task4_2'] = []
for one_input in inputs['task4']:
    answer['task4_1'].append(diag_prod_DA(**one_input))
    answer['task4_2'].append(diag_prod_AD(**one_input))

# Sparse Matrix Product

You are given two matrices of the same shape: $A$ and $B$. Matrix $A$ is full
and is given in the form of `numpy.ndarray`.

The second matrix $B$ is **sparse**. That means that the 
majority of the items are equal to $0$ except for $M$. This matrix is given
as a set of non-zero elements of this matrix in form of $3 \times M$ `numpy.ndarray` as row-column-value tuple (COO sparse matrix form):

$$
\begin{bmatrix}
r_1 & c_1 & v_1 \\
r_2 & c_2 & v_2 \\
& \vdots & \\
r_M & c_M & v_M \\
\end{bmatrix}
$$

If in this struct two items correspond to the same location, consider the latter is correct.

Write the most efficient program that calculates $AB$.

Also return the ratio between the number of multiplication operations that are needed to calculate the sparse product and the number of operations for full product.

In [20]:
import numpy as np

def sparse_matrix_product(A, B_sparse):
    n, m = A.shape
    # Ensure B_sparse is in 3xM format
    if B_sparse.shape[0] != 3:
        B_sparse = B_sparse.T
    rows, cols, vals = B_sparse
    # Convert to integers for indexing
    rows = rows.astype(int)
    cols = cols.astype(int)
    # Determine number of columns in B
    if len(cols) == 0:
        B_ncols = 0
    else:
        B_ncols = cols.max() + 1
    # Initialize result as float64 to match expected type
    res = np.zeros((n, B_ncols), dtype=np.float64)
    M = len(vals)
    # Perform sparse multiplication
    for i in range(M):
        r = rows[i]
        c = cols[i]
        v = vals[i]
        res[:, c] += A[:, r] * v
    # Calculate multiplication ratio
    mult_sparse = n * M
    mult_full = n * m * B_ncols
    # Handle division by zero
    ratio = mult_sparse / mult_full if mult_full != 0 else 0.0
    return res, ratio

In [21]:
answer['task5'] = []
for one_input in inputs['task5']:
    answer['task5'].append(sparse_matrix_product(**one_input))

In [22]:
json_tricks.dump(answer, '.answer.json', allow_nan=True)

'{"task1": {"1": [5, 9], "2": 424}, "task2": [{"__ndarray__": [[38.431067313715445], [-29.813306630467306]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[-1.184955451282823], [-3.316790194501814]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[4903475.844865944], [-14710548.047819378]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[41.27084307799329], [6.7780662459592635]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[305979176.5304247], [152338569.09953627]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[-445.23947730772977], [0.0]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[3.485726943987177], [-2.076808512572282]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[-299.6849043533943], [185.98107264567614]], "dtype": "float64", "shape": [2, 1], "Corder": true}, {"__ndarray__": [[0.39236902974294097], [4